

*#### Loading data and running it through the deep learning model *


In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
!ls gdrive/MyDrive/Apollo_20

'Copy of TIMMY_GetData+DLmodel.ipynb'   Duplicate_test_backup	      TESTDATA
'Copy of TransferLearningModel.ipynb'   Duplicate_with_unique.ipynb  'unnötig '
 DUPLICATES			        GetData+DLmodel.ipynb
 Duplicate_test			        MAIN


In [12]:
import os
os.chdir("/content/gdrive/MyDrive/Apollo_20")

In [13]:
!ls MAIN

NORMALE_BILDER	NOTES  SCREENSHOTS


In [14]:
#!mkdir normale_Bilder

In [15]:
#!cp normale_Bilder_Flo/* normale_Bilder_Random/* normale_Bilder/

In [16]:
#!cp Screenshots_Timmy/* Screenshots_Rest/* SCREENSHOTS/

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
datagen= ImageDataGenerator(
    brightness_range = (-250, 250),
    rescale=1./255,
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 3,
    validation_split=0.2) # should be test_split


In [19]:
!ls

'Copy of TIMMY_GetData+DLmodel.ipynb'   Duplicate_test_backup	      TESTDATA
'Copy of TransferLearningModel.ipynb'   Duplicate_with_unique.ipynb  'unnötig '
 DUPLICATES			        GetData+DLmodel.ipynb
 Duplicate_test			        MAIN


In [20]:
BATCH_SIZE = 32
IMG_SIZE = (320, 320)

In [21]:
train_generator = datagen.flow_from_directory(
        'MAIN',  # Main_folder
        target_size=IMG_SIZE,  # Les images seront chargée en dimension 256x2
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        subset="training")

Found 4239 images belonging to 3 classes.


In [22]:
val_generator = datagen.flow_from_directory(
        'MAIN',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        subset="validation")

Found 1059 images belonging to 3 classes.


In [23]:
test_generator = datagen.flow_from_directory(
        'TESTDATA',
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="categorical")

Found 335 images belonging to 3 classes.


In [24]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras import Sequential
from tensorflow.keras import layers

## Deep Learning Model with Transfer Learning

In [25]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [26]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [27]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [28]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [29]:
image_batch, label_batch = next(iter(train_generator))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 10, 10, 1280)


In [30]:
base_model.trainable = False

In [31]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 320, 320, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 321, 321, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 160, 160, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 160, 160, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

To generate predictions from the block of features, average over the spatial locations, using a tf.keras.layers.GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image.


In [32]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1280)


Adding final predicition layer with 3 output categories

In [33]:
prediction_layer = tf.keras.layers.Dense(3, activation = "softmax")
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 3)


In [34]:
inputs = tf.keras.Input(shape=(320, 320, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [35]:
adam_opt = optimizers.Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999)
SGD = optimizers.SGD(learning_rate=0.1)
RMSprop = optimizers.RMSprop(learning_rate=0.1)

In [36]:
base_learning_rate = 0.0001
model.compile(optimizer=SGD,
              loss= "categorical_crossentropy",
              metrics=['accuracy'])

In [37]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 320, 320, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 10, 10, 1280)      2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [38]:
# Keep getting an error when fitting the model, according to stack overflow this should fix it: 


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [39]:
initial_epochs = 1000


In [40]:
es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(train_generator,
                    epochs=initial_epochs,
                    batch_size=32, 
                    validation_data = val_generator,
                    callbacks = [es])

Epoch 1/1000
 21/133 [===>..........................] - ETA: 27:21 - loss: 16.7674 - accuracy: 0.4435

KeyboardInterrupt: ignored

### Imports 

In [ ]:
from tensorflow.keras import optimizers, callbacks
from tensorflow.keras import Sequential
from tensorflow.keras import layers


### Preprocessing 

Test and validation split 

### Architecture

Try max pooling & using multiples of 4

In [ ]:
model = Sequential()

#First Layer
model.add(layers.Conv2D(10, kernel_size=(3,3), activation='relu', input_shape=(400, 400, 3), 
                        strides = (4,4), padding='same'))

#Hidden Layers with pooling layer after each one
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3,3), activation='relu'))   
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))


#Flattening Layer
model.add(layers.Flatten())

#Output Layer
model.add(layers.Dense(3, activation='softmax')) 

model.summary() # Compare the number of parameters to the model with Dense layers

Ways to tweak the model:
    - change number of filters 
    - change kernel size
    - change from average pooling to max pooling
    - change strides size
    - change actication function (would go against recommendation from Le Wagon)

### Custom optimizer

In [ ]:
adam_opt = optimizers.Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999)
SGD = optimizers.SGD(learning_rate=0.1)
RMSprop = optimizers.RMSprop(learning_rate=0.1)

Ways to tweak the optimiser: 
- use different optimizers
- adjust learning rate and betas

### Compiler

In [ ]:
#model.compile(loss='categorical_crossentropy', 
              #optimizer= adam_opt,
              #metrics=['accuracy'])

### Model Fitting 

Note that the data is given as X, y 

In [ ]:
# Keep getting an error when fitting the model, according to stack overflow this should fix it: 


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
#es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)

#history_adam = model.fit(train_generator, 
#          batch_size=32,
#          epochs=100, 
#          validation_data = val_generator,
#          callbacks=[es])

Ways to tweak: adjust the parameters

##### Using different optimizers

In [ ]:
es = callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model.compile(loss='categorical_crossentropy', 
              optimizer= SGD,
              metrics=['accuracy'])

history_SGD = model.fit(train_generator, 
          batch_size=32,
          epochs=100, 
          validation_data = val_generator,
          callbacks=[es])

### Model Testing 

In [ ]:
model.evaluate(x_test, y_cat_test, verbose=0)

Note that we need to insert our own test data here